In [3]:
import zipfile
import databricks.koalas as ks
import sys
sys.path.insert(0, '../')
from util import util_data_access
from pyspark.sql import SparkSession
import tqdm
import pandas as pd 
import os, glob
import re
from tqdm.auto import tqdm

In [4]:
from importlib import reload

# Download and Prepare Data

In [4]:
## download spacy model 
# download data
util_data_access.download_file('en_core_web_lg.tar.gz', 'edit-pathways/spacy/en_core_web_lg.tar.gz')
! tar -xf en_core_web_lg.tar.gz
! mv en_core_web_lg-2.3.1 en_core_web_lg

In [5]:
# download data
util_data_access.download_file('newssniffer-nytimes.db.gz', 'edit-pathways/dbs/newssniffer-nytimes.db.gz')
! gunzip newssniffer-nytimes.db.gz

In [7]:
## download data
# util_data_access.download_file('edit-data.zip', 'edit-data/edit-data.zip')
# myzip = zipfile.ZipFile('edit-data.zip')
# myzip.extractall(path='edit-data')

True

# Turn on Spark

In [5]:
# turn on spark
spark = (
    SparkSession.builder \
      .config("spark.executor.instances", "50") \
      .config("spark.driver.memory", "15g") \
      .getOrCreate()
)
sc = spark.sparkContext
# set arrow to true
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")
spark.conf.set("spark.sql.execution.arrow.enabled", "false")
ks.set_option('compute.default_index_type', 'distributed')
ks.set_option('compute.shortcut_limit', 1) # Koalas will apply pyspark 

In [6]:
spark

In [8]:
import sqlite3
import sys
sys.path.append('../util')
import util_newssniffer_parsing as unp
reload(unp)

<module 'util_newssniffer_parsing' from '../util/util_newssniffer_parsing.py'>

In [10]:
sc.addFile("../util/util_newssniffer_parsing.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [11]:
import spacy
from pyspark import SparkConf, SparkContext, SparkFiles
sc.addFile("en_core_web_lg.tar.gz")
sys.path.insert(0,SparkFiles.getRootDirectory())

nlp = None
def get_nlp():
    import spacy
    global nlp
    nlp = None
    # untar
    tar_fn = 'en_core_web_lg.tar.gz'
    dirname = 'en_core_web_lg'
    if os.path.exists(tar_fn) and not os.path.exists(dirname):
        import tarfile, glob
        with tarfile.open(tar_fn) as tar:
            tar.extractall(path='en_core_web_lg')
#         for fn in glob.glob(os.path.join(dirname, '*', '*')):
#             os.rename(fn, os.path.join(dirname, os.path.basename(fn)))
        
    # unload 
    if not nlp:
        nlp = spacy.load("en_core_web_lg/en_core_web_lg-2.3.1", disable=["tagger" "ner"])
    return nlp

In [14]:
conn = sqlite3.connect('newssniffer-nytimes.db')
df = pd.read_sql('''
    SELECT * from entryversion 
    WHERE entry_id IN (SELECT distinct entry_id FROM entryversion LIMIT 200)
''', con=conn)

In [15]:
kdf = ks.from_pandas(df)

In [16]:
t = kdf['summary'].apply(unp.split_sents)

In [18]:
! pip install pycountry

Looking in indexes: https://artifactory.inf.bloomberg.com/artifactory/api/pypi/bloomberg-pypi/simple/, https://artifactory.inf.bloomberg.com/artifactory/api/pypi/python-dsp-wheels/simple


In [19]:
tdf = t.to_pandas()

Py4JJavaError: An error occurred while calling o288.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 2.0 failed 4 times, most recent failure: Lost task 5.3 in stage 2.0 (TID 55, 10.178.189.211, executor 32): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 366, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 241, in read_udfs
    arg_offsets, udf = read_single_udf(pickleSer, infile, eval_type, runner_conf)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 168, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 69, in read_command
    command = serializer._read_with_length(file)
  File "/opt/spark/python/lib/pyspark.zip/pys

# See if we can use PySpark to do the whole thing

In [26]:
tdf = kdf.to_pandas()

In [ ]:
"""struct<
    a_id: int64, 
    len_new_doc: int64, 
    len_new_sent: int64, 
    len_old_doc: int64, 
    len_old_sent: int64, 
    num_added_sents: int64, 
    num_added_words: int64, 
    num_changed_sents: int64, 
    num_removed_sents: int64, 
    num_removed_words: int64, 
    s_idx: int64, 
    s_new: list<
        item: struct<tag: string, text: string>
    >, 
    s_old: list<
        item: struct<tag: string, text: string>
    >, 
    vars_new: list<
        item: struct<tag: string, text: string>
    >, 
    vars_old: list<
        item: struct<tag: string, text: string>
    >, 
    version_nums: list<item: int64>
>"""

In [ ]:
# # -> struct<a_id: int64, len_new_doc: int64, len_new_sent: int64, len_old_doc: int64, len_old_sent: int64, num_added_sents: int64, num_added_words: int64, num_changed_sents: int64, num_removed_sents: int64, num_removed_words: int64, s_idx: int64, s_new: list<item: struct<tag: string, text: string>>, s_old: list<item: struct<tag: string, text: string>>, vars_new: list<item: struct<tag: string, text: string>>, vars_old: list<item: struct<tag: string, text: string>>, version_nums: list<item: int64>>:

In [70]:
# ConnectionOptions = dict[str, str]
DocEnum = list[dict[str, str]]
# Server = tuple[Address, ConnectionOptions]

TypeError: 'type' object is not subscriptable

In [75]:
from typing import Sequence, Dict

In [17]:
doc_enum = ks.DataFrame["tag": str, "text": str] # Sequence[Dict[str, str]]
output_type = ks.DataFrame[
    "a_id": int, 
    "len_new_doc": int, 
    "len_new_sent": int,
    "len_old_doc": int,
    "len_old_sent": int,
    "num_added_sents": int, 
    "num_added_words": int,
    "num_changed_sents": int,
    "num_removed_sents": int,
    "num_removed_words": int,
    "s_idx": int,
#     "s_new": doc_enum,
#     "s_old": doc_enum,
#     "vars_new": doc_enum,
#     "vars_old": doc_enum,
#     "version_nums": Sequence[int]
]

In [18]:
## add type inference?
def get_sentence_diff_stats_on_article(df) -> output_type:
    output = []
    initial_vers = list(unp.get_sentence_diff_stats_on_article(df, get_sentence_vars=True, ret_type='list'))
#     for stats_block_list in initial_vers:
    output_list = []
    for stats_block_dict in initial_vers[0]: #stats_block_list:
        if stats_block_dict is not None:
            output_dict = {}
            for k, v in stats_block_dict.items():
                if isinstance(v, tuple):
                    v = list(v)
                output_dict[k] = v
            output_list.append(output_dict)
#         output.append(output_list)
    return output_list

In [19]:
from pyspark.sql.functions import pandas_udf

In [20]:
g = kdf.groupby('entry_id')

In [21]:
grouped_dfs = g.apply(get_sentence_diff_stats_on_article)

In [22]:
grouped_dfs.to_pandas()

Py4JJavaError: An error occurred while calling o854.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 5.0 failed 4 times, most recent failure: Lost task 7.3 in stage 5.0 (TID 85, 10.178.189.98, executor 5): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 366, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 229, in read_udfs
    arg_offsets, udf = read_single_udf(pickleSer, infile, eval_type, runner_conf)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 168, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 69, in read_command
    command = serializer._read_with_length(file)
  File "/opt/spark/python/lib/pyspark.zip/pyspa

In [14]:
sample_df = df.loc[lambda df: df['entry_id'].isin(df['entry_id'].drop_duplicates().sample(10))]

In [16]:
grouped_dfs = (sample_df
               .groupby('entry_id')
               .apply(lambda df: unp.get_sentence_diff_stats_on_article(a=df, get_sentence_vars=True))
              )

In [30]:
sample_df.shape

(1995, 12)

In [20]:
grouped_dfs.iloc[0]

(     a_id  len_new_doc  len_old_doc  num_added_sents  num_changed_sents  \
 0  588653           41           41                0                  3   
 
    num_removed_sents                                           vars_new  \
 0                  0  ({'text': 'WASHINGTON—President Obama is leani...   
 
                                             vars_old version_nums  
 0  ({'text': 'WASHINGTON—President Obama is leani...       (1, 2)  ,
      a_id  len_new_sent  len_old_sent  num_added_words  num_removed_words  \
 0  588653            27            16               16                  5   
 
    s_idx                                              s_new  \
 0     40  [{'text': '', 'tag': ' '}, {'text': 'Mr.', 'ta...   
 
                                                s_old version_nums  
 0  [{'text': 'He', 'tag': '-'}, {'text': '', 'tag...       (1, 2)  )

# Data Manipulation

In [5]:
cache_dir = 'edit-data/article_output/'

article_page_files = glob.glob(cache_dir + '/article-output*')
diff_files         = glob.glob(cache_dir + '/diff-output-*')
version_files      = glob.glob(cache_dir + '/version-output*')

sort_by_num = lambda fs: sorted(fs, key=lambda x: int(re.search('-(\d+)-\d.json', x)[1]))
article_page_files = sort_by_num(article_page_files)
diff_files = sort_by_num(diff_files)
version_files = sort_by_num(version_files)

In [6]:
article_kdfs = None
for i in tqdm(range(1, 9)):
    kdf = ks.read_json('s3://aspangher/edit-data/article-output-%d-8.json' % i)#, index_col=0)
    if article_kdfs is None:
        article_kdfs = kdf
    else:
        article_kdfs = article_kdfs.append(kdf, ignore_index=True)

In [9]:
article_kdfs.shape 

(2791957, 2)

In [7]:
article_kdfs = article_kdfs.drop('_corrupt_record').dropna()

In [8]:
version_files = sort_by_num(version_files)

In [9]:
version_kdfs = None
for i in tqdm(range(1, 9)):
    kdf = ks.read_json('s3://aspangher/edit-data/version-output-%d-8.json' % i)#, index_col=0)
    if version_kdfs is None:
        version_kdfs = kdf
    else:
        version_kdfs = version_kdfs.append(kdf, ignore_index=True)

In [10]:
version_kdfs.shape 

(3470064, 9)

In [11]:
version_kdfs = version_kdfs.drop('_corrupt_record').dropna()

In [12]:
version_kdfs.columns

Index(['article_id', 'article_url', 'source', 'time_created', 'time_delta',
       'title', 'version', 'version_url'],
      dtype='object')

In [13]:
version_kdfs.head(1)

,article_id,article_url,source,time_created,time_delta,title,version,version_url
1,35,http://news.bbc.co.uk/go/rss/-/1/hi/world/midd...,bbc,"August 29, 2006 01:35",,Annan asks Israel to end blockade,0,https://www.newssniffer.co.uk/articles/35/vers...


In [14]:
article_kdfs.columns

Index(['html', 'url'], dtype='object')

In [29]:
full_kdfs = version_kdfs.merge(article_kdfs, left_on='version_url', right_on='url')

In [16]:
full_kdfs.head(1)

,article_id,article_url,source,time_created,time_delta,title,version,version_url,html,url
0,10001,http://news.bbc.co.uk/go/rss/-/1/hi/uk/6115912...,bbc,"November 05, 2006 21:20",1 day later,North East 'warms to devolution',2,https://www.newssniffer.co.uk/articles/10001/v...,"<p><a href=""http://news.bbc.co.uk/go/rss/-/1/h...",https://www.newssniffer.co.uk/articles/10001/v...


In [18]:
t = full_kdfs['html'].head(2)

In [21]:
s = t.iloc[0]

In [30]:
def f(s):
    return '</p><p>'.join(s.split('</p><p>')[2:])

full_kdfs['body_text'] = full_kdfs['html'].apply(f)

In [33]:
t = full_kdfs['body_text'].str.split('</p><p>')

In [38]:
from collections import defaultdict, Counter

In [39]:
c = Counter()

In [ ]:
counter = defaultdict(int)
for i in tqdm(t):
    c.update(i)

In [49]:
full_kdfs.columns

Index(['article_id', 'article_url', 'source', 'time_created', 'time_delta',
       'title', 'version', 'version_url', 'html', 'url', 'body_text'],
      dtype='object')

In [50]:
t = full_kdfs.groupby('article_id')[['version', 'body_text', 'title']]

In [52]:
t.head(1)

Py4JJavaError: An error occurred while calling o3814.collectToPython.
: org.apache.spark.SparkException: Job 32 cancelled 
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1889)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1824)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:2076)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2059)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2048)
	at org.apache.spark.util.EventLoop$$anon$1.run(EventLoop.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.runJob(DAGScheduler.scala:737)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2061)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2082)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2101)
	at org.apache.spark

KeyboardInterrupt: 

In [31]:
ks.Series

databricks.koalas.series.Series

In [34]:
full_kdfs['version_url'].isnull().value_counts()

False    3470048
Name: version_url, dtype: int64

In [ ]:
spark.stop()